In [22]:
import sys
print(sys.executable)

/Users/billdeng/anaconda3/envs/de_novo/bin/python


In [23]:
from Bio import SeqIO
import os

First, read and visualize input and output data.

In [25]:
# Get the current working directory
current_dir = os.getcwd()

# Print the current working directory
print(f"The current working directory is: {current_dir}")

The current working directory is: /Users/billdeng/PycharmProjects/genes/source


In [26]:
# Path to FASTA file
fasta_file = '../data/input.fa'

# Initialize an empty dictionary to store sequences
chromosomes = {}

# Using SeqIO.parse to read the FASTA file
for record in SeqIO.parse(fasta_file, "fasta"):
    chromosome_name = record.id
    sequence = str(record.seq)
    chromosomes[chromosome_name] = sequence

# Now, chromosomes dictionary contains all the sequences
# For example, to access the sequence for chromosome 1, you can use:
# chromosome_1_sequence = chromosomes['1']

# If you want to print out the names of all chromosomes read, you can do:
# print("Chromosomes read from the FASTA file:")
# for chromosome in chromosomes.keys():
#     print(chromosome)

In [27]:
for chromosome in chromosomes.keys():
    print(chromosome)
    # print(type(chromosome))  # This will print the type of the chromosome variable

chr1
chr2
chr3
chr4
chr5
chr6
chr7
chr8
chr9
chr10
chr11
chr12
chr13
chr14
chr15
chr16
chr17
chr18
chr19
chr20
chr21
chr22
chrX


In [28]:
print(type(chromosomes[chromosome_name]))

<class 'str'>


In [29]:
chromosome_name = "chr22"  # Just an example, replace it with your actual key
# Print the type of the object and a part of the object (if it's a string, for example)
print(f"Type: {type(chromosomes[chromosome_name])}, Beginning of Content: '{chromosomes[chromosome_name][:10]}'")
print(f"Type: {len(chromosomes[chromosome_name])}, Beginning of Content: '{chromosomes[chromosome_name][1211231:1211252]}'")

Type: <class 'str'>, Beginning of Content: 'NNNNNNNNNN'
Type: 51304566, Beginning of Content: 'NNNNNNNNNNNNNNNNNNNNN'


Use the de novo environment.

In [30]:
import os
import numpy as np
import tensorflow

In [31]:
from keras import Model
from keras import backend as K
from keras.layers import Conv2D

from keras.layers import Dense
from keras.layers import GlobalAveragePooling2D
from keras.layers import Input

In [32]:
# from janggu import *

In [9]:
from janggu.data import Bioseq
from janggu.data import Cover
from janggu.data import ReduceDim
from janggu.layers import DnaConv2D
from sklearn.metrics import roc_auc_score

In [10]:
from pkg_resources import resource_filename